In [10]:
import pandas as pd
import numpy as np

def create_feature_engineering(df):
    """
    对给定的DataFrame进行特征工程。
    """
    
    # 获取用户A和用户B的列名
    userA_cols = [col for col in df.columns if "userA" in col]
    userB_cols = [col for col in df.columns if "userB" in col]

    # 按传感器类型分组
    acc_cols = [col for col in userA_cols if "acc" in col]
    rot_cols = [col for col in userA_cols if "rot" in col]
    thm_cols = [col for col in userA_cols if "thm" in col]
    tof_cols = [col for col in userA_cols if "tof" in col]
    
    # ------------------ 1. 创建差值特征 ------------------
    for col_a, col_b in zip(userA_cols, userB_cols):
        diff_col_name = f"diff_{col_a.replace('userA_', '')}"
        df[diff_col_name] = df[col_a] - df[col_b]

    # ------------------ 2. 创建交互特征 ------------------
    # 用户A和用户B的加速度向量模长
    df["userA_acc_magnitude"] = np.sqrt(df["userA_acc_x"]**2 + df["userA_acc_y"]**2 + df["userA_acc_z"]**2)
    df["userB_acc_magnitude"] = np.sqrt(df["userB_acc_x"]**2 + df["userB_acc_y"]**2 + df["userB_acc_z"]**2)
    
    # 用户A和用户B的加速度向量差的模长
    df["diff_acc_magnitude"] = np.sqrt(df["diff_acc_x"]**2 + df["diff_acc_y"]**2 + df["diff_acc_z"]**2)
    
    # 用户A和用户B的四元数旋转角度
    # Q = w + xi + yj + zk
    # 旋转角度 θ = 2 * arccos(w)
    df["userA_rot_angle"] = 2 * np.arccos(df["userA_rot_w"].clip(-1, 1))
    df["userB_rot_angle"] = 2 * np.arccos(df["userB_rot_w"].clip(-1, 1))

    # 用户A和用户B的加速度和距离的乘积
    # 这可以表示动量或碰撞潜力
    for i in range(1, 6):
        # 简化处理，只取 ToF 传感器的均值作为近似距离
        tof_a_cols = [col for col in tof_cols if f"userA_tof_{i}" in col]
        tof_b_cols = [col for col in tof_cols if f"userB_tof_{i}" in col]
        
        df[f"userA_tof_{i}_mean"] = df[tof_a_cols].mean(axis=1)
        df[f"userB_tof_{i}_mean"] = df[tof_b_cols].mean(axis=1)

        df[f"interaction_A_acc_tof_{i}"] = df["userA_acc_magnitude"] * df[f"userA_tof_{i}_mean"]
        df[f"interaction_B_acc_tof_{i}"] = df["userB_acc_magnitude"] * df[f"userB_tof_{i}_mean"]
    
    # ------------------ 3. 聚合时序特征 ------------------
    # 找出所有可以聚合的特征列
    agg_features = [col for col in df.columns if col not in ['row_id', 'sequence_id', 'target']]
    
    agg_func = ['mean', 'std', 'min', 'max']
    
    agg_df = df.groupby('sequence_id')[agg_features].agg(agg_func)
    
    # 重命名列以避免冲突，例如：'userA_acc_x_mean', 'userA_acc_x_std'
    agg_df.columns = ['_'.join(col).strip() for col in agg_df.columns.values]
    
    # 合并聚合特征到原始数据帧
    df = df.merge(agg_df, on='sequence_id', how='left')
    
    return df

train_df = pd.read_csv("train_data.csv")
test_df = pd.read_csv("test_data.csv")

train_df_features = create_feature_engineering(train_df.copy())
test_df_features = create_feature_engineering(test_df.copy())

/var/folders/dl/0x0qfc0s6vd9wdk53j4thvnh0000gn/T/ipykernel_6566/1761748677.py:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[diff_col_name] = df[col_a] - df[col_b]
/var/folders/dl/0x0qfc0s6vd9wdk53j4thvnh0000gn/T/ipykernel_6566/1761748677.py:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[diff_col_name] = df[col_a] - df[col_b]
/var/folders/dl/0x0qfc0s6vd9wdk53j4thvnh0000gn/T/ipykernel_6566/1761748677.py:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many t

In [3]:
train_df.head()

,row_id,sequence_id,risk_score,userA_acc_x,userA_acc_y,userA_acc_z,userA_rot_w,userA_rot_x,userA_rot_y,userA_rot_z,...,userB_tof_5_v54,userB_tof_5_v55,userB_tof_5_v56,userB_tof_5_v57,userB_tof_5_v58,userB_tof_5_v59,userB_tof_5_v60,userB_tof_5_v61,userB_tof_5_v62,userB_tof_5_v63
0,SEQ_0000026_000000,SEQ_0000026,0.0,7.130933,2.732644,-6.277213,0.181571,0.897682,-0.038596,0.367841,...,45.0,40.0,90.0,59.0,64.0,55.0,38.0,39.0,39.0,36.0
1,SEQ_0000026_000001,SEQ_0000026,0.0,6.899818,2.911483,-5.995802,0.166779,0.903360,-0.020606,0.393976,...,49.0,42.0,96.0,62.0,63.0,62.0,39.0,42.0,41.0,38.0
2,SEQ_0000026_000002,SEQ_0000026,0.0,6.864221,2.899383,-6.546528,0.178188,0.932308,0.004487,0.369356,...,48.0,41.0,-1.0,60.0,64.0,60.0,36.0,38.0,40.0,38.0
3,SEQ_0000026_000003,SEQ_0000026,0.0,6.374657,2.922386,-6.705415,0.161165,0.915649,0.011314,0.353680,...,45.0,40.0,-1.0,59.0,66.0,55.0,38.0,41.0,40.0,38.0
4,SEQ_0000026_000004,SEQ_0000026,0.0,6.296517,3.019442,-6.905275,0.189661,0.930940,0.007381,0.332297,...,45.0,42.0,-1.0,60.0,67.0,54.0,39.0,37.0,41.0,38.0


In [4]:
test_df.head()

,row_id,sequence_id,userA_acc_x,userA_acc_y,userA_acc_z,userA_rot_w,userA_rot_x,userA_rot_y,userA_rot_z,userA_thm_1,...,userB_tof_5_v54,userB_tof_5_v55,userB_tof_5_v56,userB_tof_5_v57,userB_tof_5_v58,userB_tof_5_v59,userB_tof_5_v60,userB_tof_5_v61,userB_tof_5_v62,userB_tof_5_v63
0,SEQ_0000004_000000,SEQ_0000004,4.895843,0.625722,-8.696752,0.164455,-0.778759,-0.563635,-0.225310,25.399134,...,239.0,-1.0,20.0,33.0,41.0,46.0,-1.0,-1.0,13.0,21.0
1,SEQ_0000004_000001,SEQ_0000004,5.628222,-0.294839,-8.329328,0.158987,-0.780400,-0.583453,-0.225054,25.866785,...,-1.0,-1.0,34.0,65.0,102.0,122.0,112.0,197.0,27.0,-1.0
2,SEQ_0000004_000002,SEQ_0000004,5.329158,0.418915,-8.605945,0.150445,-0.748112,-0.623703,-0.220155,25.837753,...,-1.0,-1.0,72.0,103.0,126.0,138.0,115.0,-1.0,34.0,39.0
3,SEQ_0000004_000003,SEQ_0000004,4.329819,0.704076,-7.964393,0.138448,-0.741832,-0.611915,-0.211870,25.987011,...,-1.0,-1.0,40.0,121.0,135.0,144.0,120.0,-1.0,-1.0,-1.0
4,SEQ_0000004_000004,SEQ_0000004,4.924841,0.282822,-8.893566,0.121634,-0.748912,-0.614915,-0.182401,26.190262,...,-1.0,-1.0,49.0,108.0,128.0,143.0,164.0,-1.0,-1.0,-1.0


In [20]:
from sklearn.model_selection import cross_val_predict
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import HistGradientBoostingRegressor

pred = cross_val_predict(
    LinearRegression(), train_df.drop(["row_id", "sequence_id", "risk_score"], axis=1).fillna(0), train_df["risk_score"]
)
(pred - train_df["risk_score"]).abs().mean()

0.13064239721513057

In [22]:
pred = cross_val_predict(
    HistGradientBoostingRegressor(), train_df.drop(["row_id", "sequence_id", "risk_score"], axis=1), train_df["risk_score"]
)
(pred - train_df["risk_score"]).abs().mean()

0.08987084127293843

In [23]:
model = HistGradientBoostingRegressor()
model.fit(train_df.drop(["row_id", "sequence_id", "risk_score"], axis=1), train_df["risk_score"])
pred = model.predict(test_df.drop(["row_id", "sequence_id"], axis=1))

In [28]:
submit = pd.DataFrame([test_df["row_id"], pred]).T
submit.columns = ["row_id", "risk_score"]
submit.to_csv("s.csv", index=None)